# Импорты

In [1]:
!pip install ipynbname -q
!pip install lightgbm -q
!pip install tensorflow
!pip install sktime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.9/36.9 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.5/141.5 kB 12.4 MB/s eta 0:00:00


In [40]:
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb

from sktime.datatypes._panel._convert import (
    from_2d_array_to_nested,
    from_nested_to_2d_array,
)

from scipy.stats import skew, kurtosis
from scipy.fftpack import fft
from scipy.signal import find_peaks

import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import files
from google.colab import drive
import ipynbname
import shutil

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

import pickle

import datetime
import pytz

import os
from tqdm import tqdm

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Константы

In [277]:
SEED = 42

# paths
DRIVE_PATH = './gdrive/MyDrive/SiamHack/Solution'
TRAIN_PATH = os.path.join(DRIVE_PATH, 'data/train')
TEST_PATH = os.path.join(DRIVE_PATH, 'data/test')

TRAIN_TIMESERIES_PATH = os.path.join(TRAIN_PATH, 'data')
HQ_MARKUP_PATH = os.path.join(TRAIN_PATH, 'hq_markup_train.csv')
MARKUP_PATH = os.path.join(TRAIN_PATH, 'markup_train.csv')

TEST_TIMESERIES_PATH = os.path.join(TEST_PATH, 'validation')
SUBMISSION_FILE_NAME = os.path.join(TEST_PATH, 'submit.csv')

SUBS_PATH = os.path.join(DRIVE_PATH, 'submits')
NOTEBOOKS_PATH = os.path.join(DRIVE_PATH, 'notebooks')

# Вспомогательные функции

In [278]:
def set_all_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

set_all_seeds(SEED)

# Загрузка данных

Загружаем разметку

In [297]:
labels_df = pd.read_csv(MARKUP_PATH)
hq_labels_df = pd.read_csv(HQ_MARKUP_PATH)

# Высокого ли качества данные
labels_df['IsHq'] = False
hq_labels_df['IsHq'] = False

labels_df = pd.concat([labels_df, hq_labels_df], ignore_index=True)
labels_df.head()

,file_name,department_name,user_name,Некачественное ГДИС,Влияние ствола скважины,Радиальный режим,Линейный режим,Билинейный режим,Сферический режим,Граница постоянного давления,Граница непроницаемый разлом,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details,IsHq
0,2b1db5db-a184-4223-ac58-2f7a416dbab8,department1,user1,0,1,0,0,1,0,0,0,2.346340,NaN,NaN,0.494178,NaN,NaN,NaN,False
1,680641f4-eaae-4dca-9338-d9285efbda88,department2,user1,1,0,0,1,1,0,0,0,NaN,NaN,1.517418,0.659840,NaN,NaN,NaN,False
2,01fe0501-8f06-4d23-9754-05744ae95a4c,department2,user1,0,1,0,0,0,0,0,0,0.356844,NaN,NaN,NaN,NaN,NaN,NaN,False
3,4ad441ba-50ab-4520-8296-173907044b88,department1,user1,0,1,1,0,0,0,0,0,0.059301,1.344549,NaN,NaN,NaN,NaN,NaN,False
4,488841e5-cee6-4807-81a0-cb4111777b0b,department2,user1,1,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [280]:
def get_data_list(path):
  data_list = []

  # Проход по каждому файлу в папке с временными рядами
  for file_name in tqdm(os.listdir(path)[:5000]):
      file_path = os.path.join(path, file_name)

      # Чтение и парсинг данных из текстового файла
      time = []
      delta_p = []
      p_prime = []

      with open(file_path, 'r') as file:
        for line in file:
          parts = line.strip().split()
          try:
              t = float(parts[0])
              dp = float(parts[1])
              pp = float(parts[2])

              time.append(t)
              delta_p.append(dp)
              p_prime.append(pp)
          except Exception as error:
              print(f"Ошибка в файле {file_name}: {error}")
              continue

      timeseries_df = pd.DataFrame({
          'Время': time,
          'ΔP': delta_p,
          'P\'': p_prime
      })

      timeseries_df['file_name'] = file_name
      data_list.append(timeseries_df)

  return data_list

In [9]:
data_list = get_data_list(TRAIN_TIMESERIES_PATH)

100%|██████████| 5000/5000 [22:05<00:00,  3.77it/s]


In [281]:
def generate_features_for_series(series, prefix):
    features = {}

    # Базовые статистики
    features[f'{prefix}_mean'] = series.mean()
    features[f'{prefix}_std'] = series.std()
    features[f'{prefix}_min'] = series.min()
    features[f'{prefix}_max'] = series.max()
    features[f'{prefix}_median'] = series.median()
    features[f'{prefix}_skewness'] = skew(series)
    features[f'{prefix}_kurtosis'] = kurtosis(series)

    # Производные фичи
    features[f'{prefix}_range'] = features[f'{prefix}_max'] - features[f'{prefix}_min']
    features[f'{prefix}_mean_abs_change'] = np.mean(np.abs(np.diff(series)))
    features[f'{prefix}_mean_change_rate'] = np.mean(np.diff(series) / series[:-1])

    # Преобразование Фурье
    fft_values = np.abs(fft(series.to_numpy()))  # Преобразуем series в массив NumPy
    features[f'{prefix}_fft_max'] = np.max(fft_values)
    features[f'{prefix}_fft_min'] = np.min(fft_values)
    features[f'{prefix}_fft_mean'] = np.mean(fft_values)

    # Автокорреляция
    autocorr = np.correlate(series, series, mode='full')[len(series)-1:]
    features[f'{prefix}_autocorr_max'] = np.max(autocorr)
    features[f'{prefix}_autocorr_min'] = np.min(autocorr)
    features[f'{prefix}_autocorr_mean'] = np.mean(autocorr)

    # Пики и впадины
    peaks, _ = find_peaks(series)
    features[f'{prefix}_num_peaks'] = len(peaks)
    features[f'{prefix}_peaks_mean'] = np.mean(series[peaks]) if len(peaks) > 0 else 0

    # Скользящие окна
    window_size = 5
    rolling_mean = series.rolling(window=window_size).mean()
    rolling_std = series.rolling(window=window_size).std()
    features[f'{prefix}_rolling_mean_max'] = rolling_mean.max()
    features[f'{prefix}_rolling_std_max'] = rolling_std.max()

    # Разница между соседними значениями
    features[f'{prefix}_diff_max'] = np.max(np.diff(series))
    features[f'{prefix}_diff_min'] = np.min(np.diff(series))
    features[f'{prefix}_diff_mean'] = np.mean(np.diff(series))

    return features

In [298]:
def generate_features(df):
    features = {}

    delta_p_features = generate_features_for_series(df['ΔP'], 'delta_p')
    features.update(delta_p_features)
    p_prime_features = generate_features_for_series(df['P\''], 'p_prime')
    features.update(p_prime_features)

    return features

In [299]:
def generate_features_from_list(data_list):
    all_features = []

    for df in tqdm(data_list):
      try:
          features = generate_features(df)
          features['file_name'] = df['file_name'].iloc[0]
          all_features.append(features)
      except:
          continue

    return pd.DataFrame(all_features)

In [296]:
# удаляем датафреймы с < 20 строк - на них скорее всего нет паттерна
data_list = [df for df in data_list if len(df) > 20]

In [285]:
features_df = generate_features_from_list(data_list)

 64%|██████▍   | 3109/4859 [00:18<00:09, 188.21it/s]<ipython-input-281-0e10fc740ff3>:10: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features[f'{prefix}_skewness'] = skew(series)
<ipython-input-281-0e10fc740ff3>:11: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features[f'{prefix}_kurtosis'] = kurtosis(series)
100%|██████████| 4859/4859 [00:29<00:00, 165.67it/s]


In [286]:
features_df.head()

,delta_p_mean,delta_p_std,delta_p_min,delta_p_max,delta_p_median,delta_p_skewness,delta_p_kurtosis,delta_p_range,delta_p_mean_abs_change,delta_p_mean_change_rate,...,p_prime_autocorr_min,p_prime_autocorr_mean,p_prime_num_peaks,p_prime_peaks_mean,p_prime_rolling_mean_max,p_prime_rolling_std_max,p_prime_diff_max,p_prime_diff_min,p_prime_diff_mean,file_name
0,73.323438,35.218566,0.269175,113.475814,88.210292,-0.674572,-0.991273,113.206639,0.353771,0.024311,...,1.811144e+00,2.956730e+04,62,13.915451,28.055055,2.656054,3.203636,-3.043875,0.013499,54d743b1-6851-4a31-a8b5-a0ae42d0c230
1,24.439710,7.965226,0.735369,35.178863,25.944003,-0.943815,0.390227,34.443494,0.144115,0.018352,...,4.343085e-01,1.570214e+03,27,3.831073,8.112714,0.917091,0.826778,-0.441213,0.010242,34230965-3274-46cf-b253-982ad018c82b
2,48.378661,14.818338,3.686630,63.264609,53.281806,-1.238153,0.761765,59.577979,0.717807,0.039680,...,1.494389e+01,9.643916e+03,10,15.094705,25.406344,5.859100,6.112128,-5.241755,0.040975,126b4cbd-3b41-4804-89dc-94cd9acb3180
3,161.358079,94.713030,3.056770,311.790560,171.179131,-0.245459,-1.220946,308.733789,0.867230,0.014732,...,1.058287e+02,2.186962e+05,97,37.835101,60.159539,15.700614,21.270072,-31.496869,0.055992,c8be2728-2203-43f9-8701-e5d2a751b09c
4,223499.219657,57656.352608,39.363129,341444.713361,218869.895146,-0.930550,2.873873,341405.350231,961.792708,0.052258,...,5.485062e+07,1.749956e+11,89,25774.196391,104256.181775,30250.417538,24053.067317,-6720.803443,229.283117,2e5c5c6a-71d7-44a6-8bfb-86cff2bf3c39


In [300]:
def scale_and_sample(data, num_samples=20):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(np.array(data).reshape(-1, 1)).flatten()
    indices = np.linspace(0, len(scaled_data) - 1, num_samples, dtype=int)
    return scaled_data[indices]

In [301]:
def transform_data_list(data_list):
    transformed_data = []

    for df in data_list:
        file_name = df['file_name'].iloc[0]

        # Выбираем и масштабируем delta_p и p_prime
        delta_p_samples = scale_and_sample(df['ΔP'])
        p_prime_samples = scale_and_sample(df['P\''])

        # Создаем словарь с признаками
        features = {
            'file_name': file_name,
            **{f'delta_p_{i}': delta_p_samples[i] for i in range(20)},
            **{f'p_prime_{i}': p_prime_samples[i] for i in range(20)}
        }

        transformed_data.append(features)

    # Преобразуем список словарей в DataFrame
    return pd.DataFrame(transformed_data)

In [302]:
transformed_df = transform_data_list(data_list).drop(['delta_p_0'], axis=1)
transformed_df

,file_name,delta_p_1,delta_p_2,delta_p_3,delta_p_4,delta_p_5,delta_p_6,delta_p_7,delta_p_8,delta_p_9,...,p_prime_10,p_prime_11,p_prime_12,p_prime_13,p_prime_14,p_prime_15,p_prime_16,p_prime_17,p_prime_18,p_prime_19
0,54d743b1-6851-4a31-a8b5-a0ae42d0c230,0.076041,0.139773,0.214416,0.303161,0.404471,0.506038,0.597720,0.681384,0.749056,...,0.435818,0.308390,0.214527,0.158787,0.212800,0.317577,0.368790,0.231477,0.301165,0.153127
1,34230965-3274-46cf-b253-982ad018c82b,0.197457,0.363435,0.469912,0.530824,0.574856,0.614590,0.651599,0.683533,0.715831,...,0.374226,0.360369,0.346492,0.330779,0.313573,0.290135,0.274238,0.273042,0.278750,0.304461
2,126b4cbd-3b41-4804-89dc-94cd9acb3180,0.225123,0.357941,0.499813,0.566304,0.639965,0.702154,0.741809,0.779940,0.817507,...,0.618173,0.513591,0.442363,0.387373,0.305501,0.255789,0.222655,0.165930,0.156378,0.141845
3,c8be2728-2203-43f9-8701-e5d2a751b09c,0.009901,0.029703,0.108911,0.198020,0.277228,0.346535,0.396040,0.445545,0.514851,...,0.618351,0.697703,0.610974,0.358633,0.272208,0.188909,0.188898,0.614473,0.605885,0.353673
4,2e5c5c6a-71d7-44a6-8bfb-86cff2bf3c39,0.381435,0.515115,0.559329,0.578229,0.589773,0.599925,0.612543,0.622869,0.634774,...,0.106441,0.127564,0.167476,0.205670,0.267910,0.344348,0.473125,0.567254,0.683260,0.805524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4854,03206414-df2e-4193-b548-9222c1475e89,0.200328,0.343842,0.434702,0.479639,0.624439,0.702709,0.764970,0.799152,0.831418,...,0.440347,0.465864,0.430020,0.383190,0.356560,0.309959,0.320236,0.328881,0.326711,0.419809
4855,66da0152-0e66-44a2-9ffb-b8f2a4d05903,0.225975,0.369387,0.471342,0.544263,0.596599,0.640069,0.679822,0.717779,0.754774,...,0.267010,0.265430,0.265367,0.263424,0.242184,0.189901,0.119286,0.054048,0.019050,0.000000
4856,ef5e95f0-abcb-4c54-83fb-fad373616b2b,0.143448,0.211254,0.233151,0.252172,0.281085,0.321240,0.362217,0.400640,0.438652,...,0.743902,0.890938,0.979113,0.981888,0.668163,0.282605,0.043896,0.020945,0.044432,0.000000
4857,c5ba0747-50c3-46b0-839c-7238c254bd74,0.566038,0.481132,0.405660,0.405660,0.481132,0.481132,0.481132,0.566038,0.566038,...,0.030651,0.034577,0.036084,0.033512,0.047441,0.047632,0.047958,0.094891,0.458818,0.064452


# Подготовка к обучению

In [303]:
merged_df = pd.merge(transformed_df, labels_df, on='file_name', how='left')
merged_df

,file_name,delta_p_1,delta_p_2,delta_p_3,delta_p_4,delta_p_5,delta_p_6,delta_p_7,delta_p_8,delta_p_9,...,Граница постоянного давления,Граница непроницаемый разлом,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details,IsHq
0,54d743b1-6851-4a31-a8b5-a0ae42d0c230,0.076041,0.139773,0.214416,0.303161,0.404471,0.506038,0.597720,0.681384,0.749056,...,0,0,1.758848,0.910806,NaN,NaN,NaN,NaN,NaN,False
1,34230965-3274-46cf-b253-982ad018c82b,0.197457,0.363435,0.469912,0.530824,0.574856,0.614590,0.651599,0.683533,0.715831,...,0,0,1.463145,0.380994,NaN,NaN,0.685414,NaN,NaN,False
2,126b4cbd-3b41-4804-89dc-94cd9acb3180,0.225123,0.357941,0.499813,0.566304,0.639965,0.702154,0.741809,0.779940,0.817507,...,1,0,1.295397,NaN,NaN,NaN,NaN,5.990823,NaN,False
3,c8be2728-2203-43f9-8701-e5d2a751b09c,0.009901,0.029703,0.108911,0.198020,0.277228,0.346535,0.396040,0.445545,0.514851,...,0,1,2.375425,NaN,NaN,NaN,NaN,NaN,47.79251,False
4,2e5c5c6a-71d7-44a6-8bfb-86cff2bf3c39,0.381435,0.515115,0.559329,0.578229,0.589773,0.599925,0.612543,0.622869,0.634774,...,0,0,6.903400,3.907655,3.830235,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4904,03206414-df2e-4193-b548-9222c1475e89,0.200328,0.343842,0.434702,0.479639,0.624439,0.702709,0.764970,0.799152,0.831418,...,1,0,1.951166,1.015804,NaN,NaN,1.483328,5.346394,NaN,False
4905,66da0152-0e66-44a2-9ffb-b8f2a4d05903,0.225975,0.369387,0.471342,0.544263,0.596599,0.640069,0.679822,0.717779,0.754774,...,1,0,2.970026,0.990496,NaN,NaN,NaN,168.231080,NaN,False
4906,ef5e95f0-abcb-4c54-83fb-fad373616b2b,0.143448,0.211254,0.233151,0.252172,0.281085,0.321240,0.362217,0.400640,0.438652,...,0,0,1.301773,NaN,NaN,NaN,NaN,NaN,NaN,False
4907,c5ba0747-50c3-46b0-839c-7238c254bd74,0.566038,0.481132,0.405660,0.405660,0.481132,0.481132,0.481132,0.566038,0.566038,...,0,0,2.061197,NaN,0.199872,NaN,NaN,NaN,NaN,False


In [310]:
time_features = [
    'delta_p_1', 'delta_p_2', 'delta_p_3', 'delta_p_4',
       'delta_p_5', 'delta_p_6', 'delta_p_7', 'delta_p_8', 'delta_p_9',
       'delta_p_10', 'delta_p_11', 'delta_p_12', 'delta_p_13', 'delta_p_14',
       'delta_p_15', 'delta_p_16', 'delta_p_17', 'delta_p_18', 'delta_p_19',
       'p_prime_0', 'p_prime_1', 'p_prime_2', 'p_prime_3', 'p_prime_4',
       'p_prime_5', 'p_prime_6', 'p_prime_7', 'p_prime_8', 'p_prime_9',
       'p_prime_10', 'p_prime_11', 'p_prime_12', 'p_prime_13', 'p_prime_14',
       'p_prime_15', 'p_prime_16', 'p_prime_17', 'p_prime_18', 'p_prime_19'
]
cat_targets = ['Некачественное ГДИС',
       'Влияние ствола скважины', 'Радиальный режим', 'Линейный режим',
       'Билинейный режим', 'Сферический режим', 'Граница постоянного давления',
       'Граница непроницаемый разлом']
num_targets = ['Влияние ствола скважины_details',
       'Радиальный режим_details', 'Линейный режим_details',
       'Билинейный режим_details', 'Сферический режим_details',
       'Граница постоянного давления_details',
       'Граница непроницаемый разлом_details']

In [311]:
X_train_timeseries = from_2d_array_to_nested(merged_df[time_features])
X_train_timeseries

,0
0,0 0.076041 1 0.139773 2 0.214416 3...
1,0 0.197457 1 0.363435 2 0.469912 3...
2,0 0.225123 1 0.357941 2 0.499813 3...
3,0 0.009901 1 0.029703 2 0.108911 3...
4,0 0.381435 1 0.515115 2 0.559329 3...
...,...
4904,0 0.200328 1 0.343842 2 0.434702 3...
4905,0 0.225975 1 0.369387 2 0.471342 3...
4906,0 0.143448 1 0.211254 2 0.233151 3...
4907,0 0.566038 1 0.481132 2 0.405660 3...


In [312]:
y_train = merged_df[cat_targets]

## TimeSeriesForestClassifier

In [307]:
from sktime.classification.interval_based import TimeSeriesForestClassifier

In [308]:
X_train, X_val, y_train, y_val = train_test_split(X_train_timeseries, y_train, test_size=0.1)

Обучение: (модель не поддерживает мультиклассовую классификацию)

In [309]:
forest_models = []
for target in cat_targets:
  forest = TimeSeriesForestClassifier()
  forest.fit(X_train, y_train[target])

  forest_models.append(forest)

Валидация:

In [313]:
for i, target in enumerate(cat_targets):
  y_pred_forest = forest_models[i].predict(X_val)
  print(f"F1_score on {target}:", f1_score(y_val[target], y_pred_forest))

F1_score on Некачественное ГДИС: 0.4421052631578947
F1_score on Влияние ствола скважины: 0.8982857142857142
F1_score on Радиальный режим: 0.8025889967637541
F1_score on Линейный режим: 0.4834123222748815
F1_score on Билинейный режим: 0.5277777777777778
F1_score on Сферический режим: 0.14084507042253522
F1_score on Граница постоянного давления: 0.03636363636363636
F1_score on Граница непроницаемый разлом: 0.0851063829787234


In [314]:
def get_models_size(model):
  model_bytes = pickle.dumps(model)
  model_size = len(model_bytes)
  print(f"Размер моделей: {8 * (model_size) // 1024} килобайт")

In [315]:
get_models_size(forest_models[0])

Размер моделей: 101413 килобайт


# Регрессия

In [318]:
from sklearn.multioutput import MultiOutputRegressor

In [319]:
# Объединяем нагенеренные фичи и разметку
merged_df = pd.merge(features_df, labels_df, on='file_name', how='left')

In [320]:
merged_df

,delta_p_mean,delta_p_std,delta_p_min,delta_p_max,delta_p_median,delta_p_skewness,delta_p_kurtosis,delta_p_range,delta_p_mean_abs_change,delta_p_mean_change_rate,...,Граница постоянного давления,Граница непроницаемый разлом,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details,IsHq
0,73.323438,35.218566,0.269175,113.475814,88.210292,-0.674572,-0.991273,113.206639,0.353771,0.024311,...,0,0,1.758848,0.910806,NaN,NaN,NaN,NaN,NaN,False
1,24.439710,7.965226,0.735369,35.178863,25.944003,-0.943815,0.390227,34.443494,0.144115,0.018352,...,0,0,1.463145,0.380994,NaN,NaN,0.685414,NaN,NaN,False
2,48.378661,14.818338,3.686630,63.264609,53.281806,-1.238153,0.761765,59.577979,0.717807,0.039680,...,1,0,1.295397,NaN,NaN,NaN,NaN,5.990823,NaN,False
3,161.358079,94.713030,3.056770,311.790560,171.179131,-0.245459,-1.220946,308.733789,0.867230,0.014732,...,0,1,2.375425,NaN,NaN,NaN,NaN,NaN,47.79251,False
4,223499.219657,57656.352608,39.363129,341444.713361,218869.895146,-0.930550,2.873873,341405.350231,961.792708,0.052258,...,0,0,6.903400,3.907655,3.830235,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4904,66.441132,21.740907,3.991672,88.252627,75.267585,-1.190249,0.418396,84.260954,0.991305,0.042239,...,1,0,1.951166,1.015804,NaN,NaN,1.483328,5.346394,NaN,False
4905,123.848282,37.122008,13.412610,165.535120,131.010479,-0.914218,0.208082,152.122510,1.635726,0.030629,...,1,0,2.970026,0.990496,NaN,NaN,NaN,168.231080,NaN,False
4906,16.962320,9.590735,0.110049,29.631035,14.359289,0.103974,-1.488865,29.520986,0.506847,0.274843,...,0,0,1.301773,NaN,NaN,NaN,NaN,NaN,NaN,False
4907,124.109171,6.031620,102.615404,139.362812,123.415824,-0.094982,1.472820,36.747408,2.415533,0.005531,...,0,0,2.061197,NaN,0.199872,NaN,NaN,NaN,NaN,False


In [321]:
X = merged_df.drop(num_targets, axis=1).drop(['file_name', 'department_name', 'user_name'], axis=1)
y = merged_df[num_targets].fillna(-1)

In [322]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=SEED)

In [323]:
X_train

,delta_p_mean,delta_p_std,delta_p_min,delta_p_max,delta_p_median,delta_p_skewness,delta_p_kurtosis,delta_p_range,delta_p_mean_abs_change,delta_p_mean_change_rate,...,p_prime_diff_mean,Некачественное ГДИС,Влияние ствола скважины,Радиальный режим,Линейный режим,Билинейный режим,Сферический режим,Граница постоянного давления,Граница непроницаемый разлом,IsHq
2865,104.873425,46.100809,2.109644,140.124340,129.008661,-1.294028,-0.064629,138.014696,2.409941,0.084411,...,0.368847,0,1,0,0,0,0,0,1,False
4618,131.588534,13.997838,18.063715,140.321498,137.295647,-3.974729,26.155676,122.257783,0.864493,0.035554,...,-1.298883,0,1,1,0,0,0,0,1,False
4096,161.626242,85.363537,3.263828,255.134583,185.568855,-0.526187,-1.180152,251.870756,0.787096,0.013999,...,0.071003,0,1,1,0,0,0,0,1,False
2016,108.770319,18.024847,0.113092,128.593962,112.114807,-2.962847,11.813443,128.480870,0.431107,0.087547,...,0.022248,0,0,1,1,1,0,0,0,False
2195,39.194246,5.118634,4.440845,42.925981,40.906070,-3.811939,19.060346,38.485136,0.258290,0.021381,...,-0.029328,0,1,1,0,0,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4426,103.286583,22.769987,3.104217,148.286067,100.724250,-0.543470,1.655173,145.181850,0.453693,0.020843,...,0.000672,0,1,0,0,1,1,0,0,False
466,4.443738,5.158135,0.002766,18.805218,2.106804,1.267264,0.470892,18.802452,0.042274,0.031058,...,0.013166,0,1,1,1,0,0,0,0,False
3092,68.365335,43.608863,1.109309,145.929536,70.344027,0.134263,-1.338402,144.820228,0.787066,0.029385,...,0.222289,0,1,0,0,1,0,0,0,False
3772,1.715991,0.672873,0.195947,3.868038,1.641372,0.807780,1.109929,3.672092,0.049068,0.046527,...,0.004062,0,1,1,0,0,0,0,1,False


In [325]:
lgb_model = lgb.LGBMRegressor(learning_rate=0.01, n_estimators=200, max_depth=8)
model = MultiOutputRegressor(lgb_model)
model.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003420 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11447
[LightGBM] [Info] Number of data points in the train set: 4859, number of used features: 54
[LightGBM] [Info] Start training from score 1.178317
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005568 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11447
[LightGBM] [Info] Number of data points in the train set: 4859, number of used features: 54
[LightGBM] [Info] Start training from score 0.269223
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of te

MultiOutputRegressor(estimator=LGBMRegressor(learning_rate=0.01, max_depth=8,
                                             n_estimators=200))

In [326]:
get_models_size(model)

Размер моделей: 26775 килобайт


In [327]:
y_pred = model.predict(X_test)
y_pred

array([[ 9.38752075e-01, -8.29949943e-01, -2.80170538e-01,
         2.42916479e-02, -9.60168122e-01,  1.09577544e+00,
         1.35415117e+00],
       [ 1.27629245e+00, -8.29949943e-01, -9.41075804e-01,
        -5.82098390e-02, -9.60168122e-01,  1.09577544e+00,
         1.35415117e+00],
       [ 4.83971642e+00,  3.35389932e+00,  2.86131693e+00,
         3.16438625e+00, -9.60168122e-01,  1.09577544e+00,
         1.35415117e+00],
       [ 1.35696723e+00, -8.29949943e-01, -9.41075804e-01,
         5.73102413e-01, -9.60168122e-01,  1.09577544e+00,
         1.35415117e+00],
       [ 1.47965569e+00, -8.29949943e-01,  5.68861923e-01,
        -9.28880688e-01, -9.60168122e-01,  1.09577544e+00,
         1.35415117e+00],
       [ 1.39579614e+00, -8.29949943e-01, -9.41075804e-01,
         6.06265218e-01, -9.60168122e-01,  1.09577544e+00,
         1.35415117e+00],
       [ 1.93527019e+00,  4.30068405e-01,  6.04313647e-01,
         3.84639701e-01, -9.60168122e-01,  1.09577544e+00,
         2.4038182

# Cабмит

## Классификация

In [328]:
labels_df = pd.read_csv(SUBMISSION_FILE_NAME)
labels_df

,file_name,Некачественное ГДИС,Влияние ствола скважины,Радиальный режим,Линейный режим,Билинейный режим,Сферический режим,Граница постоянного давления,Граница непроницаемый разлом,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details
0,283bbcbc-ee90-4a6a-ae06-8392104a2c10,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,cabe112d-93ae-4bb2-9585-a36d3bc31349,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3eaf5015-fe2c-4f54-87c7-cd91142e19f7,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,612e4ad5-98a9-4a53-b034-d14d02305126,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5bf5e5d7-5ad2-49ba-8bcc-719831ad60bc,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,6fe55825-442f-4a73-a659-50346c8096d2,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,10f9be1f-9ccb-4fa2-ad74-69d53941a6a4,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,903787df-01b6-4308-b037-11c7ba81eb4e,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498,3c264f8a-df09-4beb-b64f-879c515e88b0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [329]:
test_data_list = get_data_list(TEST_TIMESERIES_PATH)

100%|██████████| 500/500 [00:03<00:00, 129.88it/s]


In [330]:
test_features_df = generate_features_from_list(test_data_list)

 34%|███▍      | 171/500 [00:00<00:01, 175.72it/s]/usr/local/lib/python3.11/dist-packages/scipy/stats/_stats_py.py:1177: RuntimeWarning: Mean of empty slice.
  mean = a.mean(axis, keepdims=True)
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/scipy/stats/_stats_py.py:1287: RuntimeWarning: Mean of empty slice.
  mean = a.mean(axis, keepdims=True)
 42%|████▏     | 211/500 [00:01<00:01, 185.00it/s]/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=a

In [331]:
test_features_df

,delta_p_mean,delta_p_std,delta_p_min,delta_p_max,delta_p_median,delta_p_skewness,delta_p_kurtosis,delta_p_range,delta_p_mean_abs_change,delta_p_mean_change_rate,...,p_prime_autocorr_min,p_prime_autocorr_mean,p_prime_num_peaks,p_prime_peaks_mean,p_prime_rolling_mean_max,p_prime_rolling_std_max,p_prime_diff_max,p_prime_diff_min,p_prime_diff_mean,file_name
0,15.521514,9.386963,3.073813,27.696223,15.725371,0.017003,-1.718810,24.622411,4.545339,0.290420,...,531.580627,2640.858898,4,22.121049,26.319933,22.324270,32.733835,-45.607974,0.173737,b16485d1-e26a-47ad-aae7-84a8ca81a9c0
1,16.507195,11.211658,0.103564,37.696763,15.793893,0.266153,-1.168494,37.593198,0.414907,0.087104,...,8.944236,3529.602546,35,9.255847,15.753302,8.261458,20.308594,-19.530882,0.067893,92f4ca66-ccb1-41ad-a491-ec8a11ce00dc
2,33.282738,16.989293,0.744010,52.692088,40.467494,-0.579424,-1.201606,51.948079,0.212213,0.018338,...,2.263085,6722.934683,43,6.550877,18.482985,1.024808,2.114635,-2.151098,0.007057,43aa1d72-b972-4bcb-ae15-36a76fc35de1
3,73.173899,38.277464,1.699334,156.146403,68.102861,0.387294,-0.831255,154.447069,0.891624,0.038222,...,64.224586,125417.063888,23,30.319294,307.144066,163.680432,285.908601,-98.477699,0.155291,deb5be7f-31cd-483c-bbde-6035b39be602
4,2.153597,0.185986,1.857072,2.340170,2.174808,-0.501630,-0.922635,0.483098,0.096620,0.047957,...,1.262343,3.396855,1,0.924357,0.929121,0.763423,0.796713,-1.682875,-0.228429,62bd7e28-d3fb-402d-a46d-54a79d93945b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,37.965979,12.626068,0.450463,51.242824,41.314816,-0.889789,-0.064997,50.792361,0.403821,0.084407,...,0.260435,4593.447414,12,9.684703,12.318422,2.001235,2.584513,-2.119267,-0.013804,0342e696-55be-4022-a146-3590cebd9d53
492,223.724706,38.154422,70.056642,258.659401,234.413585,-1.943952,3.902115,188.602759,1.042147,0.007397,...,14.421359,34407.654545,17,18.733895,65.618099,12.298639,8.723734,-9.927343,-0.017370,93b53614-6cf2-4ea0-ad4a-f70eb7fd74bc
493,70.640853,24.415252,0.850769,86.125205,82.426297,-1.683247,1.298789,85.274436,0.357436,0.021984,...,3.891177,11460.205819,39,3.478752,51.031459,4.250784,2.418627,-3.048153,0.007709,85cfaa4f-2e02-49da-95e5-36fbd1515914
494,41.835614,26.035455,0.064109,73.456001,51.591871,-0.488270,-1.339577,73.391892,0.455850,0.048085,...,2.492127,17499.775221,1,11.456378,47.857018,1.511671,1.013838,-0.401265,0.308352,b85fde97-d36c-4967-9775-90f9d3c8b9df


In [332]:
# Отбираем только те фичи, где есть хотя бы 20 наблюдений
filtered_test_data_list = [df for df in test_data_list if len(df) > 20]
# Остальные считаем некачественными
bad_test_data_list = [df for df in test_data_list if len(df) <= 20]

print(len(filtered_test_data_list), len(bad_test_data_list))

480 20


In [333]:
# Получаем все уникальные file_name из bad_test_data_list
combined_df = pd.concat(bad_test_data_list, ignore_index=True)
unique_file_names_df = combined_df[['file_name']].drop_duplicates().reset_index(drop=True)
unique_file_names_df

,file_name
0,b16485d1-e26a-47ad-aae7-84a8ca81a9c0
1,62bd7e28-d3fb-402d-a46d-54a79d93945b
2,89de8613-1f50-4d99-a50d-b0683c5d9d59
3,31377dd8-a873-4940-b55c-91261fffe2c3
4,29d8bc1f-5d65-4cd5-8bbb-13147f387b76
5,72fe9a43-2efa-48df-bb46-c295e8adbeb6
6,d1b251a7-97bb-4f70-a939-8d83ff214016
7,2c09a44d-a129-4eb3-b0af-b2e7465d8685
8,9e93f1fb-8b0b-429b-83d4-2d06435634c0
9,287db032-b8aa-4608-b389-2fe60f0a6a54


In [334]:
# Запоняем как некачественное ГДИС те наблюдения, которые содержатся в unique_file_names_df
labels_df.loc[labels_df['file_name'].isin(unique_file_names_df['file_name']), 'Некачественное ГДИС'] = 1

In [335]:
transformed_test = transform_data_list(filtered_test_data_list).drop(['delta_p_0'], axis=1)
transformed_test

,file_name,delta_p_1,delta_p_2,delta_p_3,delta_p_4,delta_p_5,delta_p_6,delta_p_7,delta_p_8,delta_p_9,...,p_prime_10,p_prime_11,p_prime_12,p_prime_13,p_prime_14,p_prime_15,p_prime_16,p_prime_17,p_prime_18,p_prime_19
0,92f4ca66-ccb1-41ad-a491-ec8a11ce00dc,0.023759,0.060221,0.101737,0.141253,0.198509,0.229391,0.257536,0.277663,0.379979,...,0.298086,0.341590,0.284092,0.393480,0.460061,0.365209,0.469963,0.500041,0.403115,0.327171
1,43aa1d72-b972-4bcb-ae15-36a76fc35de1,0.061826,0.109614,0.169314,0.238088,0.333806,0.441435,0.562961,0.664351,0.734260,...,0.369044,0.266820,0.212783,0.192659,0.174343,0.136963,0.135198,0.199558,0.112018,0.102335
2,deb5be7f-31cd-483c-bbde-6035b39be602,0.136369,0.186470,0.220003,0.255866,0.300715,0.333460,0.070090,0.321831,0.398325,...,0.054427,0.046474,0.050719,0.057858,0.065940,0.079711,0.092219,0.102915,0.118352,0.139434
3,557b78db-ac5e-4b03-bf7f-4a05d9fa5f76,0.842656,0.907827,0.926099,0.936352,0.945589,0.953609,0.959192,0.964978,0.970155,...,0.020844,0.015991,0.019353,0.007333,0.004993,0.007482,0.005271,0.006774,0.013510,0.000000
4,07fba59f-ade5-4995-9257-775b106ea341,0.034412,0.068076,0.098934,0.128951,0.150084,0.186927,0.210959,0.232093,0.283523,...,0.399423,0.534567,0.521185,0.393124,0.819609,0.375891,0.611120,1.000000,0.584083,0.697955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,0342e696-55be-4022-a146-3590cebd9d53,0.248847,0.376995,0.474476,0.531421,0.587426,0.630843,0.679920,0.739090,0.784672,...,0.972555,0.815651,0.672485,0.555013,0.457708,0.383699,0.317975,0.175444,0.096908,0.000000
476,93b53614-6cf2-4ea0-ad4a-f70eb7fd74bc,0.350098,0.591114,0.683234,0.728579,0.763294,0.795517,0.817098,0.841834,0.860946,...,0.183495,0.178085,0.147804,0.118153,0.084560,0.080405,0.071546,0.065920,0.075446,0.000000
477,85cfaa4f-2e02-49da-95e5-36fbd1515914,0.131992,0.267198,0.460401,0.709380,0.887275,0.934179,0.942386,0.948405,0.954086,...,0.005697,0.004811,0.006258,0.006152,0.006593,0.006324,0.004544,0.004809,0.017200,0.036754
478,b85fde97-d36c-4967-9775-90f9d3c8b9df,0.006609,0.017197,0.046081,0.102096,0.217727,0.356727,0.497774,0.586357,0.664837,...,0.155248,0.214217,0.265924,0.322352,0.377195,0.483939,0.579017,0.717793,0.838960,1.000000


In [336]:
X_test_timeseries = from_2d_array_to_nested(transformed_test.drop('file_name', axis=1))

In [337]:
y_preds = []
for i, target in enumerate(cat_targets):
  y_pred = forest_models[i].predict(X_test_timeseries)
  y_preds.append(y_pred)

In [338]:
len(y_preds[0])

480

In [339]:
labels_df

,file_name,Некачественное ГДИС,Влияние ствола скважины,Радиальный режим,Линейный режим,Билинейный режим,Сферический режим,Граница постоянного давления,Граница непроницаемый разлом,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details
0,283bbcbc-ee90-4a6a-ae06-8392104a2c10,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,cabe112d-93ae-4bb2-9585-a36d3bc31349,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3eaf5015-fe2c-4f54-87c7-cd91142e19f7,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,612e4ad5-98a9-4a53-b034-d14d02305126,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5bf5e5d7-5ad2-49ba-8bcc-719831ad60bc,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,6fe55825-442f-4a73-a659-50346c8096d2,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,10f9be1f-9ccb-4fa2-ad74-69d53941a6a4,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,903787df-01b6-4308-b037-11c7ba81eb4e,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498,3c264f8a-df09-4beb-b64f-879c515e88b0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [340]:
y_preds_df = pd.DataFrame(y_preds).T  # Транспонируем, чтобы каждая колонка соответствовала target
y_preds_df.columns = cat_targets  # Называем колонки по именам целевых фичей

# Добавляем file_name из transformed_test
y_preds_df['file_name'] = transformed_test['file_name'].values

# Объединяем submit с y_preds_df по file_name
submit = pd.merge(labels_df.drop(cat_targets, axis=1), y_preds_df, on='file_name', how='left')

In [341]:
submit

,file_name,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details,Некачественное ГДИС,Влияние ствола скважины,Радиальный режим,Линейный режим,Билинейный режим,Сферический режим,Граница постоянного давления,Граница непроницаемый разлом
0,283bbcbc-ee90-4a6a-ae06-8392104a2c10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,cabe112d-93ae-4bb2-9585-a36d3bc31349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3eaf5015-fe2c-4f54-87c7-cd91142e19f7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,612e4ad5-98a9-4a53-b034-d14d02305126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5bf5e5d7-5ad2-49ba-8bcc-719831ad60bc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,6fe55825-442f-4a73-a659-50346c8096d2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
496,10f9be1f-9ccb-4fa2-ad74-69d53941a6a4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
497,903787df-01b6-4308-b037-11c7ba81eb4e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
498,3c264f8a-df09-4beb-b64f-879c515e88b0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [342]:
# Новый порядок столбцов
new_columns_order = ['file_name'] + cat_targets + [col for col in submit.columns if col not in cat_targets and col != 'file_name']
submit = submit[new_columns_order]
submit

,file_name,Некачественное ГДИС,Влияние ствола скважины,Радиальный режим,Линейный режим,Билинейный режим,Сферический режим,Граница постоянного давления,Граница непроницаемый разлом,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details
0,283bbcbc-ee90-4a6a-ae06-8392104a2c10,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,cabe112d-93ae-4bb2-9585-a36d3bc31349,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3eaf5015-fe2c-4f54-87c7-cd91142e19f7,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,612e4ad5-98a9-4a53-b034-d14d02305126,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5bf5e5d7-5ad2-49ba-8bcc-719831ad60bc,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,6fe55825-442f-4a73-a659-50346c8096d2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,10f9be1f-9ccb-4fa2-ad74-69d53941a6a4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,903787df-01b6-4308-b037-11c7ba81eb4e,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498,3c264f8a-df09-4beb-b64f-879c515e88b0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [343]:
submit.isna().sum()

,0
file_name,0
Некачественное ГДИС,20
Влияние ствола скважины,20
Радиальный режим,20
Линейный режим,20
Билинейный режим,20
Сферический режим,20
Граница постоянного давления,20
Граница непроницаемый разлом,20
Влияние ствола скважины_details,0


In [344]:
submit.loc[(submit['Некачественное ГДИС'] != 0) & (submit['Некачественное ГДИС'] != 1), 'Некачественное ГДИС'] = 1

In [345]:
for target in cat_targets:
  submit.loc[(submit[target] != 0) & (submit[target] != 1), target] = 0

In [346]:
submit.isna().sum()

,0
file_name,0
Некачественное ГДИС,0
Влияние ствола скважины,0
Радиальный режим,0
Линейный режим,0
Билинейный режим,0
Сферический режим,0
Граница постоянного давления,0
Граница непроницаемый разлом,0
Влияние ствола скважины_details,0


# Регрессия

In [347]:
# Объединяем нагенеренные фичи и разметку
merged_df = pd.merge(submit, test_features_df, on='file_name', how='left')

In [348]:
X = merged_df.drop(num_targets, axis=1).drop(['file_name'], axis=1)

In [349]:
X['IsHq'] = False

In [350]:
y_pred = model.predict(X)
y_pred

array([[-7.08149757e-01, -8.29949943e-01,  1.03614670e-01, ...,
         1.86413033e-01,  1.91004400e+02,  7.06509346e+02],
       [ 4.15014261e-01,  4.46868079e-01,  2.91164867e-01, ...,
         1.57522143e-01,  1.09577544e+00,  3.51405625e+02],
       [ 9.26948262e-02,  2.14709766e-01,  1.70723953e-01, ...,
         6.52214312e-03,  1.09577544e+00,  4.89690273e+02],
       ...,
       [ 4.33495040e-01,  7.70501210e-01,  2.53030554e-01, ...,
         1.74327381e-01,  1.09577544e+00,  3.83367824e+02],
       [-2.83226398e-01,  3.72902986e-01,  2.76396886e-01, ...,
         1.77392086e-01,  1.09577544e+00,  4.89690273e+02],
       [ 2.00580201e-01,  1.10923432e+00,  8.01702763e-01, ...,
         3.87242855e-01,  1.09577544e+00,  7.90049801e+02]])

In [351]:
y_preds_df = pd.DataFrame(y_pred)
y_preds_df.columns = num_targets  # Называем колонки по именам целевых фичей

y_preds_df['file_name'] = merged_df['file_name'].values

y_preds_df

,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details,file_name
0,-0.708150,-0.829950,0.103615,1.229827,0.186413,191.004400,706.509346,283bbcbc-ee90-4a6a-ae06-8392104a2c10
1,0.415014,0.446868,0.291165,1.202330,0.157522,1.095775,351.405625,cabe112d-93ae-4bb2-9585-a36d3bc31349
2,0.092695,0.214710,0.170724,1.054433,0.006522,1.095775,489.690273,3eaf5015-fe2c-4f54-87c7-cd91142e19f7
3,-0.068385,-0.214974,0.244943,1.034517,-0.129754,1.095775,1.354151,612e4ad5-98a9-4a53-b034-d14d02305126
4,0.344435,0.915279,0.335006,1.296483,0.317024,1.095775,1.354151,5bf5e5d7-5ad2-49ba-8bcc-719831ad60bc
...,...,...,...,...,...,...,...,...
495,0.421796,0.643613,0.329136,1.296230,0.305912,1.095775,1.354151,6fe55825-442f-4a73-a659-50346c8096d2
496,0.627029,1.731879,0.880008,1.785764,2.273384,-1.597288,353.805433,10f9be1f-9ccb-4fa2-ad74-69d53941a6a4
497,0.433495,0.770501,0.253031,1.170924,0.174327,1.095775,383.367824,903787df-01b6-4308-b037-11c7ba81eb4e
498,-0.283226,0.372903,0.276397,1.169161,0.177392,1.095775,489.690273,3c264f8a-df09-4beb-b64f-879c515e88b0


In [352]:
# Объединяем submit с y_preds_df по file_name
submit_final = pd.merge(submit.drop(num_targets, axis=1), y_preds_df, on='file_name', how='left')

In [353]:
submit_final

,file_name,Некачественное ГДИС,Влияние ствола скважины,Радиальный режим,Линейный режим,Билинейный режим,Сферический режим,Граница постоянного давления,Граница непроницаемый разлом,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details
0,283bbcbc-ee90-4a6a-ae06-8392104a2c10,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.708150,-0.829950,0.103615,1.229827,0.186413,191.004400,706.509346
1,cabe112d-93ae-4bb2-9585-a36d3bc31349,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.415014,0.446868,0.291165,1.202330,0.157522,1.095775,351.405625
2,3eaf5015-fe2c-4f54-87c7-cd91142e19f7,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.092695,0.214710,0.170724,1.054433,0.006522,1.095775,489.690273
3,612e4ad5-98a9-4a53-b034-d14d02305126,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.068385,-0.214974,0.244943,1.034517,-0.129754,1.095775,1.354151
4,5bf5e5d7-5ad2-49ba-8bcc-719831ad60bc,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.344435,0.915279,0.335006,1.296483,0.317024,1.095775,1.354151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,6fe55825-442f-4a73-a659-50346c8096d2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.421796,0.643613,0.329136,1.296230,0.305912,1.095775,1.354151
496,10f9be1f-9ccb-4fa2-ad74-69d53941a6a4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.627029,1.731879,0.880008,1.785764,2.273384,-1.597288,353.805433
497,903787df-01b6-4308-b037-11c7ba81eb4e,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.433495,0.770501,0.253031,1.170924,0.174327,1.095775,383.367824
498,3c264f8a-df09-4beb-b64f-879c515e88b0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.283226,0.372903,0.276397,1.169161,0.177392,1.095775,489.690273


In [354]:
for feature in cat_targets:
    # Находим соответствующий столбец с окончанием _details
    details_feature = feature + '_details'

    if details_feature in submit_final.columns:
        # Заменяем значения на NaN, где target = 0
        submit_final.loc[submit_final[feature] == 0, details_feature] = 0

In [355]:
# Преобразование всех cat_targets в int
for feature in cat_targets:
    if feature in submit_final.columns:  # Проверяем, существует ли столбец
        submit_final[feature] = submit_final[feature].astype(int)

In [356]:
labels_df

,file_name,Некачественное ГДИС,Влияние ствола скважины,Радиальный режим,Линейный режим,Билинейный режим,Сферический режим,Граница постоянного давления,Граница непроницаемый разлом,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details
0,283bbcbc-ee90-4a6a-ae06-8392104a2c10,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,cabe112d-93ae-4bb2-9585-a36d3bc31349,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3eaf5015-fe2c-4f54-87c7-cd91142e19f7,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,612e4ad5-98a9-4a53-b034-d14d02305126,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5bf5e5d7-5ad2-49ba-8bcc-719831ad60bc,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,6fe55825-442f-4a73-a659-50346c8096d2,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,10f9be1f-9ccb-4fa2-ad74-69d53941a6a4,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,903787df-01b6-4308-b037-11c7ba81eb4e,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498,3c264f8a-df09-4beb-b64f-879c515e88b0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [357]:
submit_final

,file_name,Некачественное ГДИС,Влияние ствола скважины,Радиальный режим,Линейный режим,Билинейный режим,Сферический режим,Граница постоянного давления,Граница непроницаемый разлом,Влияние ствола скважины_details,Радиальный режим_details,Линейный режим_details,Билинейный режим_details,Сферический режим_details,Граница постоянного давления_details,Граница непроницаемый разлом_details
0,283bbcbc-ee90-4a6a-ae06-8392104a2c10,0,1,1,0,0,0,0,0,-0.708150,-0.829950,0.0,0.000000,0.000000,0.0,0.0
1,cabe112d-93ae-4bb2-9585-a36d3bc31349,0,1,1,0,0,0,0,0,0.415014,0.446868,0.0,0.000000,0.000000,0.0,0.0
2,3eaf5015-fe2c-4f54-87c7-cd91142e19f7,0,1,0,0,1,0,0,0,0.092695,0.000000,0.0,1.054433,0.000000,0.0,0.0
3,612e4ad5-98a9-4a53-b034-d14d02305126,0,1,0,0,0,0,0,0,-0.068385,0.000000,0.0,0.000000,0.000000,0.0,0.0
4,5bf5e5d7-5ad2-49ba-8bcc-719831ad60bc,0,1,1,0,0,1,0,0,0.344435,0.915279,0.0,0.000000,0.317024,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,6fe55825-442f-4a73-a659-50346c8096d2,0,1,1,0,0,0,0,0,0.421796,0.643613,0.0,0.000000,0.000000,0.0,0.0
496,10f9be1f-9ccb-4fa2-ad74-69d53941a6a4,0,1,1,0,0,0,0,0,0.627029,1.731879,0.0,0.000000,0.000000,0.0,0.0
497,903787df-01b6-4308-b037-11c7ba81eb4e,0,1,1,0,0,0,0,0,0.433495,0.770501,0.0,0.000000,0.000000,0.0,0.0
498,3c264f8a-df09-4beb-b64f-879c515e88b0,0,1,0,0,0,0,0,0,-0.283226,0.000000,0.0,0.000000,0.000000,0.0,0.0


## Скачивание

In [358]:
def save_submit():
    current_time = datetime.datetime.now(pytz.timezone('Asia/Krasnoyarsk'))
    submit_final.to_csv(os.path.join(SUBS_PATH, 'sub_' + current_time.strftime("%d_%B_%H:%M") + '.csv'), index=False)

In [359]:
def save_notebook():
    current_time = datetime.datetime.now(pytz.timezone('Asia/Krasnoyarsk'))
    new_name = 'nb_' + current_time.strftime("%d_%B_%H:%M") + '.ipynb'

    notebook_path = os.path.join(DRIVE_PATH, 'Solution sktime.ipynb')

    destination_path = f"{NOTEBOOKS_PATH}/{new_name}.ipynb"
    shutil.copy(notebook_path, destination_path)

In [360]:
save_submit()
# save_notebook()

# Auto downloading
submit_final.to_csv('submit.csv', index=False)
files.download('submit.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>